In [1]:
import pandas as pd
import numpy as np
# import torch
import re
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
# import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# import arabic_reshaper
# from bidi.algorithm import get_display
# import matplotlib.font_manager as fm
import csv


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Data

In [3]:
import pandas as pd
merged = pd.read_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/StructuredNewsDataset_2022_2023-06-15.csv", encoding="utf-8", lineterminator='\n')
# merged = pd.read_csv("/content/drive/MyDrive/Datasets/TestEventExtractionDataWithSentiments.csv", encoding="utf-8", lineterminator='\n')
if "Unnamed: 0" in merged.columns:
  del merged['Unnamed: 0']
elif "index" in merged.columns:
  del merged['index']
merged['DateTime'] = pd.to_datetime(merged['DateTime'])
merged = merged.sort_values('DateTime')
merged.reset_index(drop=True, inplace=True)
print(merged.shape)
merged.head(2)

(62054, 18)


,News_ID,Headline,DateTime,Url,News,CC_wo_Stopwords_punc,Source,Category,main_cat,Sub_Category,Keywords_CC,sentiment,NER_Persons,NER_Orgs,NER_Loc,Embedding_Vector,Sector Words,SectMatched Words
0,2345,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟,2017-08-05,https://www.bbc.com/urdu/regional-40825452,یہ تحریر پہلی مرتبہ بی بی سی اردو کی ویب سائٹ ...,مہندرا اینڈ محمد تقسیم الگ تحریر مرتبہ بی بی ا...,BBC,پاکستان,pakistan,Economic,"['مہندرا', 'کیشو', 'غلام', 'محمد', 'اینڈ', 'ان...",1.0,"['مہندرا', 'قارئین', 'بیٹن', 'چبھن', 'محمد', '...",['مسلم'],"['پاکستان', 'والا']",[ 5.68543494e-01 1.73920870e-01 -6.24002457e-...,"['ٹریڈنگ', 'ڈالر', 'ارب']",3
1,2372,تقسیم کی ایک نامکمل محبت کی کہانی,2017-08-15,https://www.bbc.com/urdu/regional-40912412,یہ تحریر پہلی مرتبہ 15 اگست سنہ 2017 کو بی بی ...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,BBC,پاکستان,pakistan,NotAssigned,"['عصمت', 'جیتو', 'کملا', 'پٹیل', 'عورتوں', 'ام...",1.0,"['میموائر', 'برباد', 'عصمت انھی', 'انھی', 'مین...",[],"['پاکستان', 'لاہور', 'ہندوستان', 'راولپنڈی', '...",[ 4.60777760e-01 -4.43428397e-01 -5.55949569e-...,NaN,0


In [ ]:
if merged['NER_Orgs'][0]:
  print("Yes")

Yes


In [ ]:
print(f"Shape Before Duplicate Removal: {merged.shape}")
merged.drop_duplicates(inplace=True)
print(".......................................")
print(f"Shape After Duplicate Removal: {merged.shape}")
# merged.to_csv("/content/drive/MyDrive/Datasets/EventsExtracted/BBC_Nawaiwaqt2022_1DayEvents_All.csv", index=False)


Shape Before Duplicate Removal: (62054, 16)
.......................................
Shape After Duplicate Removal: (62054, 16)


### Fix Data Formats

In [4]:
import pandas as pd
import ast

# Define a function to convert string representation of lists back to actual lists
def convert_str_to_list(string):
    try:
        # Use ast.literal_eval to safely evaluate the string as a list
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        # If the string cannot be converted to a list, return it as is
        return string

In [5]:
def string_to_array(string_value):
    # Remove the square brackets from the string
    string_value = string_value.strip('[]')

    # Convert the string to a numpy array
    array_value = np.fromstring(string_value, sep=' ')

    # Reshape the array to its original shape
    return array_value.reshape((768,))

merged['Embedding_Vector'] = merged['Embedding_Vector'].apply(string_to_array)


In [5]:
import ast
# cols = ['Keywords_CC',	'CC_tokenized',	'NER_Persons', 'NER_Organizations']
cols = ['Keywords_CC',	'NER_Persons', 'NER_Orgs', 'NER_Loc', 'Sector Words']
for col in cols:
  merged[col] = merged[col].apply(convert_str_to_list)

In [18]:
# Apply the conversion function only to columns with list values
for column in merged.columns:
    print(column, type(merged[column][0]))

# Now, only the columns with list values have been restored to actual lists
# The other columns with non-list values remain unchanged

News_ID <class 'numpy.int64'>
Headline <class 'str'>
DateTime <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Url <class 'str'>
News <class 'str'>
CC_wo_Stopwords_punc <class 'str'>
Source <class 'str'>
Category <class 'str'>
main_cat <class 'str'>
Sub_Category <class 'str'>
Keywords_CC <class 'list'>
sentiment <class 'numpy.float64'>
NER_Persons <class 'list'>
NER_Orgs <class 'list'>
NER_Loc <class 'list'>
Embedding_Vector <class 'str'>
Sector Words <class 'list'>
SectMatched Words <class 'numpy.int64'>


# Majority Voting and Purity Calculation

In [8]:
from collections import Counter

def majority_voting(elements):
    if not elements:
        return None

    counter = Counter(elements)
    majority_count = max(counter.values())
    majority_elements = [element for element, count in counter.items() if count == majority_count]

    if len(majority_elements) == 1:
        return majority_elements[0]
    else:
        return None

elements = ['pak', 'biz', 'pak', 'ent', 'biz', 'world', 'biz', 'pak']
majority = majority_voting(elements)

if majority is not None:
    print(f"The majority element is: {majority}")
else:
    print("No majority element found.")


No majority element found.


In [9]:
from collections import Counter

def calculate_majority_purity(elements):
    if not elements:
        return None

    counter = Counter(elements)
    # print(f"counter: {counter}")
    majority_count = max(counter.values())
    # print(f"majoity_count: {majority_count}")
    majority_elements = [element for element, count in counter.items() if count == majority_count]
    # print(f"majority_elements: {majority_elements}")

    majority_percentage = (majority_count / len(elements))
    return majority_percentage

elements = ['pak', 'pak', 'pak', 'pak', 'biz', 'world', 'pak', 'pak']
purity = calculate_majority_purity(elements)

if purity is not None:
    print(f"The purity of the majority element is: {purity}")
else:
    print("No majority element found.")


The purity of the majority element is: 0.75


#### Sentiment Majority Voting

In [10]:
'''
if pos = neg --> neu
if pos = neg = neu --> neu
if pos = neu --> pos
if neg = neu --> neg
'''
from collections import Counter

def find_majority(votes):
    vote_count = Counter(votes)
    top_two = vote_count.most_common(2)
    if len(top_two)>1 and top_two[0][1] == top_two[1][1]:
        if top_two[0][0] != 1 and top_two[1][0] != 1:
          # It is a tie so neutral
          return 1     # 0 --> neg, 1 --> neu, 2 --> pos
        elif top_two[0][0] == 1:
          return top_two[1][0]
        elif top_two[1][0] == 1:
          return top_two[0][0]
    return top_two[0][0]

find_majority([0,0,0,2,2,1,1,1]) # It is a tie

0

# Category Priority Wise

In [11]:
merged['main_cat'].unique()

array(['pakistan', 'world', 'business'], dtype=object)

In [12]:
def select_priority_category(categories):
#     priority_mapping = {
#         'business': 1,
#         'science': 2,
#         'health': 3,
#         'crime': 4,
#         'sports': 5,
#         'entertainment': 6,
#         'pakistan': 7,
#         'world': 8,
#         'other': 9
#     }
    priority_mapping = {
        'business': 1,
        'pakistan': 2,
        'world': 3,
        'science': 4,
        'health': 5,
        'sports': 6,
        'crime': 7,
        'entertainment': 8,
        'other': 9
    }

    selected_category = None
    selected_priority = float('inf')

    for category in categories:
        priority = priority_mapping.get(category, float('inf'))
        if priority < selected_priority:
            selected_category = category
            selected_priority = priority

    return selected_category

def calculate_purity(selected_category, categories):
    selected_count = categories.count(selected_category)
    total_count = len(categories)
    purity = selected_count / total_count
    return purity
# Example usage
categories = ['world', 'sports']
# categories = ['pakistan', 'world', 'sports', 'sports', 'world', 'business', 'sports']
major_category = select_priority_category(categories)
purity = calculate_purity(major_category, categories)

print("Selected Major Category:", major_category)
print("Purity:", purity)


Selected Major Category: world
Purity: 0.5


# Common Functions

In [13]:
def cosine(x, y):
    x = x.flatten()
    y = y.flatten()
    return 1 - distance.cosine(x, y)

def intersection(lst1, lst2):
    # Use of hybrid method
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

def intersection_rate(words_list_, words_list1_):
    # Citation calc
    min_ = words_list_.copy()
    max_ = words_list1_.copy()
    if len(max_) < len(min_):
        min_ = words_list1_.copy()
        max_ = words_list_.copy()

    intersection_list = intersection(min_, max_)
    intersection_rate_ = len(intersection_list) / len(min_)

    return intersection_rate_

def similarity_score(sim_cos, citation):
    return sim_cos * 0.8 + citation * 0.2

# Create Data Subsets to Reduce Time Cost
dividing based on months

In [ ]:
# Create a new column for the month
merged['Month'] = merged['DateTime'].dt.month
# Now, group the DataFrame by 'Month'
subsets_by_month = dict(tuple(merged.groupby('Month')))

# Access subsets for each month
for month, subset in subsets_by_month.items():
    # You can use subset for each month's data
    print(f"Data for month {month}:")
    print(subset.shape)

In [ ]:
merged.tail(1)

In [ ]:
%%time
# Access subsets for specific months (e.g., January, February, and March)
data = subsets_by_month.get(10)
print(data.shape)
# Compute BERT embeddings for all articles
data['Embeddings'] = data['CC_wo_Stopwords_punc'].apply(lambda desc: np.mean(get_roberta_embeddings(desc), axis=0))
data.to_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Oct2022.csv", index=False)

In [ ]:
%%time
data = subsets_by_month.get(11)
print(data.shape)
# Compute BERT embeddings for all articles
data['Embeddings'] = data['CC_wo_Stopwords_punc'].apply(lambda desc: np.mean(get_roberta_embeddings(desc), axis=0))
data.to_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Nov2022.csv", index=False)

In [ ]:
%%time
data = subsets_by_month.get(12)
print(data.shape)
# Compute BERT embeddings for all articles
data['Embeddings'] = data['CC_wo_Stopwords_punc'].apply(lambda desc: np.mean(get_roberta_embeddings(desc), axis=0))
data.to_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Dec2022.csv", index=False)

# Merging Datasets

In [ ]:
import os
# import pandas as pd
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')

# Set the folder path where your CSV files are located
folder_path = '/content/drive/MyDrive/Datasets/SubsetsToDeleteLater/'

# List CSV files in the folder
csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

# Initialize an empty DataFrame to store the merged data
merged_df = pd.DataFrame()

# Loop through the CSV files and concatenate them
for csv_file in csv_files:
    file_path = os.path.join(folder_path, csv_file)
    df = pd.read_csv(file_path)
    merged_df = pd.concat([merged_df, df], ignore_index=True)

# Print the first few rows of the merged DataFrame
print(merged_df.shape)


(103169, 22)


In [ ]:
merged_df.columns

Index(['Headline', 'DateTime', 'Url', 'News', 'Source', 'Category', 'Combined',
       'Combined_Cleaned', 'CC_wo_Stopwords', 'Combined_wo_Stopwords',
       'CC_wo_Stopwords_punc', 'main_cat', 'Sub_Category', 'Keywords_CC', 'id',
       'sentiment', 'CC_tokenized', 'NER_Persons', 'NER_Orgs', 'NER_Loc',
       'Embedding', 'Embedding_Vector'],
      dtype='object')

In [ ]:
merged_df = pd.DataFrame()

In [ ]:
merged_df.to_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/NewsData2022_2023-06-15.csv", index=False)

In [ ]:
d1 = pd.read_csv("/content/drive/MyDrive/Datasets/Processed_News/missing_data.csv", encoding="utf-8", lineterminator='\n')
d2 = pd.read_csv("/content/drive/MyDrive/Datasets/Processed_News/NewsDataWithSentsNER.csv", encoding="utf-8", lineterminator='\n')
# d3 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Mar2022.csv", encoding="utf-8", lineterminator='\n')
# d4 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Apr2022.csv", encoding="utf-8", lineterminator='\n')
# d5 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_May2022.csv", encoding="utf-8", lineterminator='\n')
# d6 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Jun2022.csv", encoding="utf-8", lineterminator='\n')
# d7 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Jul2022.csv", encoding="utf-8", lineterminator='\n')
# d8 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Aug2022.csv", encoding="utf-8", lineterminator='\n')
# d9 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Sep2022.csv", encoding="utf-8", lineterminator='\n')
# d10 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Oct2022.csv", encoding="utf-8", lineterminator='\n')
# d11 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Nov2022.csv", encoding="utf-8", lineterminator='\n')
# d12 = pd.read_csv("/content/drive/MyDrive/Datasets/News_2022/EventExtractionDataWithEmbeddings_Dec2022.csv", encoding="utf-8", lineterminator='\n')

In [ ]:
merged = pd.concat([d1,d2])
if "Unnamed: 0" in merged.columns:
  del merged['Unnamed: 0']
elif "index" in merged.columns:
  del merged['index']

merged['DateTime'] = pd.to_datetime(merged['DateTime'])
merged = merged.sort_values('DateTime')
merged.reset_index(drop=True, inplace=True)
print(merged.shape)
merged.to_csv("/content/drive/MyDrive/Datasets/Processed_News/NewsData2022_2023-06-15.csv", index=False)
merged.head(2)

(103169, 20)


,Headline,DateTime,Url,News,Source,Category,Combined,Combined_Cleaned,CC_wo_Stopwords,Combined_wo_Stopwords,CC_wo_Stopwords_punc,main_cat,Sub_Category,Keywords_CC,id,sentiment,CC_tokenized,NER_Persons,NER_Orgs,NER_Loc
0,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟,2017-08-05,https://www.bbc.com/urdu/regional-40825452,یہ تحریر پہلی مرتبہ بی بی سی اردو کی ویب سائٹ ...,BBC,پاکستان,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟ یہ ...,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا یہ ت...,مہندرا اینڈ محمد تقسیم الگ تحریر مرتبہ بی بی ا...,مہندرا اینڈ محمد تقسیم الگ کیا؟ تحریر مرتبہ بی...,مہندرا اینڈ محمد تقسیم الگ تحریر مرتبہ بی بی ا...,pakistan,NotAssigned,"['مہندرا', 'کیشو', 'غلام', 'محمد', 'اینڈ', 'ان...",2345,1.0,"['مہندرا', 'اینڈ', 'محمد', 'کو', 'تقسیم', 'نے'...","['مہندرا', 'قارئین', 'بیٹن', 'چبھن', 'محمد', '...",['مسلم'],"['پاکستان', 'والا']"
1,تقسیم کی ایک نامکمل محبت کی کہانی,2017-08-15,https://www.bbc.com/urdu/regional-40912412,یہ تحریر پہلی مرتبہ 15 اگست سنہ 2017 کو بی بی ...,BBC,پاکستان,تقسیم کی ایک نامکمل محبت کی کہانی یہ تحریر پہل...,تقسیم کی ایک نامکمل محبت کی کہانی یہ تحریر پہل...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,تقسیم نامکمل محبت کہانی تحریر مرتبہ 15 اگست سن...,pakistan,NotAssigned,"['عصمت', 'جیتو', 'کملا', 'پٹیل', 'عورتوں', 'ام...",2372,1.0,"['تقسیم', 'کی', 'ایک', 'نامکمل', 'محبت', 'کی',...","['میموائر', 'برباد', 'عصمت انھی', 'انھی', 'مین...",[],"['پاکستان', 'لاہور', 'ہندوستان', 'راولپنڈی', '..."


In [ ]:
merged.to_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/NewsData2022_2023-06-15.csv", index=False)

In [ ]:
merged.columns

Index(['Url', 'Headline', 'Category', 'Source', 'News', 'DateTime', 'main_cat',
       'Combined', 'Combined_Cleaned', 'CC_wo_Stopwords',
       'Combined_wo_Stopwords', 'CC_wo_Stopwords_punc', 'Keywords_CC',
       'CC_tokenized', 'NER_Persons', 'NER_Organizations', 'sentiment',
       'Month', 'Embeddings'],
      dtype='object')

#### Extra

In [ ]:
article = merged['CC_wo_Stopwords_punc'][0]
vectors = get_roberta_embeddings(article)
print(vectors)
if len(vectors) > 0:
    query_embedding = np.mean(vectors, axis=0)
    print("*************************")
    print(len(vectors))
    print(query_embedding)

In [ ]:
data.head(2)

In [ ]:
e1 = data['Embeddings'][36093]
e2 = data['Embeddings'][37092]

In [ ]:
sim_cos = cosine(e1, e2)
print(sim_cos)

In [ ]:
if e1 != 0:
  print("yes")

In [ ]:
# source1_articles = df[df['Source'] == max_source]

# Get the number of articles for each source
# num_source1_articles = len(source1_articles)
num_source1_articles = 2
source1_embeddings = np.zeros((num_source1_articles, 768), dtype='float32')
for j, source1_embedding in enumerate(source1_embeddings):
  print(source1_embedding)
  # # sim_cos = cosine(query_embedding, source1_embedding)
  print(sim_cos)
# citation = intersection_rate(words, source1_word_lists[j])
# score = similarity_score(sim_cos, citation)
# source1_similarities.append(citation)

In [ ]:
source1_embedding

In [ ]:
sim_cos

In [ ]:
data.head(2)

In [ ]:
merged.to_csv("/content/drive/MyDrive/Datasets/TestEventExtractionDataWithSentimentsEmbeddings.csv", index=False)

# Event Extraction

### Event Extraction Function

In [14]:
def event_extraction(df):

  added_urls = set()
  for ind, row in df.iterrows():
    sources, urls, tareekhs, desc = [], [], [], []
    cats, sim_score, sentiments, sub_cat = [], [], [], []
    nerper, keywords = row['NER_Persons'], row['Keywords_CC']
    nerorg, nerloc = row['NER_Orgs'], row['NER_Loc']
    url = row['Url']
    sim_score.append(-1)
    sentiments.append(row['sentiment'])
    cats.append(row['main_cat'])
    urls.append(url)
    sources.append(row['Source'])
    tareekhs.append(row['DateTime'])
    desc.append(row['News'])
    sub_cat.append(row['Sub_Category'])
    if row['Sub_Category'] != 'NotAssigned':
      sect_words = row['Sector Words']
    else:
      sect_words = []

    # Skip url if already part of a previous event
    if url not in added_urls:
      vector1 = row['Embedding_Vector']
      # Get Words list of the main article
      article = row['CC_wo_Stopwords_punc']
      article  = re.sub(r'(\d+)(\D)', r'\1 \2', article)
      article  = re.sub(r'(\d+)', r' \1', article)
      words = article.replace('nbsp', '').split()

      # Get a subset of indexes greater than the index of currrent article
      all_indexes = df.index.tolist()
      subset_indexes = [index for index in all_indexes if index > ind]

      # Compare with all the other articles in the dataframe
      # similarities = []
      for index in subset_indexes:
        article2 = df.iloc[index]['CC_wo_Stopwords_punc']
        article2  = re.sub(r'(\d+)(\D)', r'\1 \2', article2)
        article2  = re.sub(r'(\d+)', r' \1', article2)
        words2 = article2.replace('nbsp', '').split()

        vector2 = df.iloc[index]['Embedding_Vector']

        # Find Similarity/Citation
        citation = intersection_rate(words, words2)
        sim_cos = cosine(vector1, vector2)
        score = similarity_score(sim_cos, citation)


        if score > 0.8 and (citation > 0.5):
          new_url = df.iloc[index]['Url']
          if new_url not in added_urls:   # Check if the url has already been identified as part of another event
            urls.append(new_url)
            added_urls.add(new_url)     # Add the new url to the added_urls set to avoid repetition
            sources.append(df.iloc[index]['Source'])
            tareekhs.append(df.iloc[index]['DateTime'])
            desc.append(df.iloc[index]['News'])
            cats.append(df.iloc[index]['main_cat'])
            sentiments.append(df.iloc[index]['sentiment'])
            sub_cat.append(df.iloc[index]['Sub_Category'])
            keywords += df.iloc[index]['Keywords_CC']
            if df.iloc[index]['NER_Persons']:     # Check if NER list is empty, if Empty then skip
              nerper += df.iloc[index]['NER_Persons']
            if df.iloc[index]['NER_Orgs']:
              nerorg += df.iloc[index]['NER_Orgs']
            if df.iloc[index]['NER_Loc']:
              nerloc += df.iloc[index]['NER_Loc']
            if df.iloc[index]['Sub_Category'] != 'NotAssigned':
              sect_words += df.iloc[index]['Sector Words']
            if -1 in sim_score:
              sim_score.remove(-1)
            sim_score.append(score)
            # print(f"Citation: {citation}\nCosine: {sim_cos}\nScore: {score}")
            # print(urls)
            # print("----------------------------------------")

      url_category_dict = {}
      # url_subcat_dict = {}
      url_senti_dict = {}
      for url, category in zip(urls, cats):
        if url not in url_category_dict:
            url_category_dict[url] = category

      # for url, sub in zip(urls, sub_cat):
      #   if url not in url_subcat_dict:
      #       url_subcat_dict[url] = sub

      for url, sentiment in zip(urls, sentiments):
        if url not in url_senti_dict:
            url_senti_dict[url] = sentiment

      # Separate the unique URLs and categories from the dictionary
      urls = list(url_category_dict.keys())
      cats = list(url_category_dict.values())
      # sub_cat = list(url_subcat_dict.values())
      sub_cat = list(set(sub_cat))
      sentiments = list(url_senti_dict.values())
      sources = list(set(sources))
      desc = list(set(desc))
      keywords = list(set(keywords))
      nerper = list(set(nerper))
      nerorg = list(set(nerorg))
      nerloc = list(set(nerloc))
      sect_words = list(set(sect_words))
      maj_cat = majority_voting(cats)
      priority = select_priority_category(cats)
      purity = calculate_purity(priority, cats)
      maj_purity = calculate_majority_purity(cats)
      maj_senti = find_majority(sentiments)
      new_row = {'Event': df.iloc[ind]['Headline'], 'Sources': sources, 'Urls': urls, 'Description': desc, 'Articles': len(urls),
                  'Categories': cats, 'Sub_Category': sub_cat, 'Sector Words': sect_words,'Major_Cat': maj_cat, 'Priority_cat': priority, 'Priority_Purity': purity, 'Maj_Purity': maj_purity,
                  'Sentiment': maj_senti, 'Keywords': keywords, 'Similarity_Scores': sim_score, 'NER_Persons': nerper, 'NER_Orgs': nerorg, 'NER_Loc': nerloc, 'Start Date': start_date, 'End Date': end_date}
      global events
      events = pd.concat([events, pd.DataFrame([new_row])], ignore_index=True)

## 1 Day Event Extraction

### All News

In [29]:
%%time
import numpy as np
import re
import torch
# Find the oldest and latest dates
oldest_date = merged['DateTime'].min()
latest_date = merged['DateTime'].max()

print("Oldest Date:", oldest_date)
print("Latest Date:", latest_date)

dates = list(merged['DateTime'].unique())
dates = np.sort(dates)
print(len(dates), dates[0:5])

# List of column names
cols = ['Event', 'Sources', 'Urls', 'Articles', 'Start Date', 'End Date']
# Create an empty dataframe to store all the events
global events
events = pd.DataFrame(columns=cols)

for date in dates:
  global start_date
  global end_date
  start_date = date
  end_date = date

  df = merged[merged['DateTime']==date]
  df.reset_index(drop=True, inplace=True)

  print(f"START_DATE: {start_date}, END_DATE: {end_date}")
  event_extraction(df)

# events_all1 = events
# events_all1.to_csv("/content/drive/MyDrive/Datasets/PEP_Demo/PEP_Demo2_1DayEvents_withCitations_065threshold.csv", index=False)
events.to_csv("/content/drive/MyDrive/Datasets/Processed_NewsData/1Day_Events.csv", index=False)

print(f"Columns: {events.columns}\nShape: {events.shape}")
events.head(3)

START_DATE: 2022-01-29T00:00:00.000000000, END_DATE: 2022-01-29T00:00:00.000000000
START_DATE: 2022-01-30T00:00:00.000000000, END_DATE: 2022-01-30T00:00:00.000000000
START_DATE: 2022-01-31T00:00:00.000000000, END_DATE: 2022-01-31T00:00:00.000000000
START_DATE: 2022-02-01T00:00:00.000000000, END_DATE: 2022-02-01T00:00:00.000000000
START_DATE: 2022-02-02T00:00:00.000000000, END_DATE: 2022-02-02T00:00:00.000000000
START_DATE: 2022-02-03T00:00:00.000000000, END_DATE: 2022-02-03T00:00:00.000000000
START_DATE: 2022-02-04T00:00:00.000000000, END_DATE: 2022-02-04T00:00:00.000000000
START_DATE: 2022-02-05T00:00:00.000000000, END_DATE: 2022-02-05T00:00:00.000000000
START_DATE: 2022-02-06T00:00:00.000000000, END_DATE: 2022-02-06T00:00:00.000000000
START_DATE: 2022-02-07T00:00:00.000000000, END_DATE: 2022-02-07T00:00:00.000000000
START_DATE: 2022-02-08T00:00:00.000000000, END_DATE: 2022-02-08T00:00:00.000000000
START_DATE: 2022-02-09T00:00:00.000000000, END_DATE: 2022-02-09T00:00:00.000000000
STAR

,Event,Sources,Urls,Articles,Start Date,End Date,Description,Categories,Sub_Category,Sector Words,Major_Cat,Priority_cat,Priority_Purity,Maj_Purity,Sentiment,Keywords,Similarity_Scores,NER_Persons,NER_Orgs,NER_Loc
0,مہندرا اینڈ محمد کو تقسیم نے کیسے الگ کیا؟,[BBC],[https://www.bbc.com/urdu/regional-40825452],1,2017-08-05,2017-08-05,[یہ تحریر پہلی مرتبہ بی بی سی اردو کی ویب سائٹ...,[pakistan],[Economic],"[ٹریڈنگ, ارب, ڈالر]",pakistan,pakistan,1.0,1.0,1.0,"[محمد, کمپنی, ہندو, قائل, اینڈ, چبھن, انڈیا, ت...",[-1],"[محمد, بیٹن, مہندرا, دہلی, قارئین, چبھن]",[مسلم],"[پاکستان, والا]"
1,تقسیم کی ایک نامکمل محبت کی کہانی,[BBC],[https://www.bbc.com/urdu/regional-40912412],1,2017-08-15,2017-08-15,[یہ تحریر پہلی مرتبہ 15 اگست سنہ 2017 کو بی بی...,[pakistan],[NotAssigned],[],pakistan,pakistan,1.0,1.0,1.0,"[آپریشن, دباؤ, کتاب, جیتو, محبت, تقسیم, کملا, ...",[-1],"[جیتو, مخالفت, میموائر, گننے, عصمت انھی, 'عصمت...",[],"[راولپنڈی, ہندوستان, لاہور, سرینگر, پاکستان]"
2,پاکستانی تاریخ کی ’تاریک ترین‘ رات جب ’ملک بچا...,[BBC],[https://www.bbc.com/urdu/pakistan-45805505],1,2018-10-10,2018-10-10,[سات اکتوبر 1958 کو لگائے جانے والے پاکستان کے...,[pakistan],[Economic],"[قدر, رقم]",pakistan,pakistan,1.0,1.0,1.0,"[جنرل, صدر, سہروردی, لا, جعفر, اعظم, محلاتی, ش...",[-1],"[علی خان, بہتیرا, مرزا میر, علی, فقدان, محمد, ...","[اقبال, کالعدم]","[ہندوستان, کراچی, برطانیہ, کوڑا, کوئٹہ, پاکستا..."


In [ ]:
events[events['Articles']>8]

,Event,Sources,Urls,Articles,Start Date,End Date,Description,Categories,Sub_Category,Major_Cat,Priority_cat,Priority_Purity,Maj_Purity,Sentiment,Keywords,Similarity_Scores,NER_Persons,NER_Orgs,NER_Loc
42151,سپریم کورٹ نے انتخابات التواء کیس کا فیصلہ محف...,"[Jang, Nawaiwaqt]","[https://jang.com.pk/news/1211576, https://jan...",10,2023-04-03,2023-04-03,[سپریم کورٹ آف پاکستان نے پی ٹی آئی رہنما فوا...,"[pakistan, pakistan, pakistan, pakistan, pakis...",[NotAssigned],pakistan,pakistan,1.0,1.0,1.0,"[سپریم, حکومت, عدالتوں, جواب, ملبہ, عمر, راستے...","[0.859943099910181, 0.8082123762838245, 0.8145...","[اظہر, الجواب, جواد, لکھی, ۔عمران, کھینچ, بریف...","[مسلم, تحریک انصاف, پیپلز, تحریکِ انصاف, تحریک...","[پشاور, بوٹا, خوا, والا, کوٹ, شاہراہِ, اسلام آ..."


In [ ]:
events['Sub_Category'].apply(tuple).unique()

array([('NotAssigned',), ('Cement',), ('Economic',),
       ('Cement, Economic',), ('Cement, Economic', 'NotAssigned'),
       ('Cement, Economic', 'NotAssigned', 'Economic'),
       ('NotAssigned', 'Economic'), ('Cement', 'NotAssigned'),
       ('Oil & Gas',), ('Cement, Oil & Gas',),
       ('NotAssigned', 'Oil & Gas')], dtype=object)

In [ ]:
events_all1.to_csv("/content/drive/MyDrive/Datasets/Processed_News/1DayEvents_2022-2023.csv", index=False)

### Analysis

In [ ]:
events['Urls'][89589]

['https://www.bbc.com/urdu/articles/cpwydggg567o',
 'https://www.bbc.com/urdu/articles/c72jlj8rendo']

In [ ]:
events['NER_Persons'][1214]

['ردعمل', 'کوکاکولا', 'ویلیو', 'کولا', 'خطاب', 'کرسٹیانو', 'پیو', 'مشروب']

#### Duplicate Removal

In [ ]:
events_all1['Urls'] = events_all1['Urls'].apply(tuple)

print(f"Shape Before Duplicate Removal: {events_all1.shape}")
# events_all1.drop_duplicates(inplace=True)
events_all1.drop_duplicates(subset='Urls' ,inplace=True)
events_all1.reset_index(drop=True)
print(".......................................")
print(f"Shape After Duplicate Removal: {events_all1.shape}")


indsub1, indsub2 = [], []
articles = np.sort(events_all1['Articles'].unique())
for art in articles:
  sub1 = events_all1[events_all1['Articles']==art]
  sub2 = events_all1[events_all1['Articles']>art]
  for index, row in sub1.iterrows():
    url = row['Urls']
    # print(type(url), url)
    for ind, row2 in sub2.iterrows():
      urls = row2['Urls']
      if set(url).issubset(set(urls)):
        indsub1.append(index)
        indsub2.append(ind)
  # print(art, sub1.shape, sub2.shape)

print(f"indeces dropped: {len(set(indsub1))}")

# Drop rows by indices
events_all1.drop(indsub1, inplace=True)

# Reset index after dropping rows
events_all1.reset_index(drop=True, inplace=True)

# events_all1.to_csv("/content/drive/MyDrive/Datasets/PEP_Demo/PEP_Demo2_1DayEvents_withCitations_065threshold.csv", index=False)

# Print the resulting DataFrame
print(events_all1.shape)

Shape Before Duplicate Removal: (2330, 16)
.......................................
Shape After Duplicate Removal: (2330, 16)
indeces dropped: 0
(2330, 16)


##### Extra

In [ ]:
articles = np.sort(events_all1['Articles'].unique())
articles

array([1, 2, 3, 4, 5, 6, 8], dtype=object)

In [ ]:
yo = 9
sub1 = events_all1[events_all1['Articles']==yo]
sub2 = events_all1[events_all1['Articles']>yo]

indsub14, indsub24 = [], []
for index, row in sub1.iterrows():
  url = row['Urls']
  # print(type(url), url)
  for ind, row2 in sub2.iterrows():
    urls = row2['Urls']
    for u in url:
      if u in urls:
        print(f"YES, url: {url} at index {index} in {urls} at index: {ind}")
        print("************************************")
        indsub14.append(index)
        indsub24.append(ind)

In [ ]:
c = events_all1[events_all1['Articles']==8]
c.shape

(1, 16)

In [ ]:
d = list(c['Urls'])
d

[('https://jang.com.pk/news/1224244',
  'https://jang.com.pk/news/1224232',
  'https://jang.com.pk/news/1224308',
  'https://jang.com.pk/news/1224287',
  'https://jang.com.pk/news/1224148',
  'https://jang.com.pk/news/1224315',
  'https://jang.com.pk/news/1224360',
  'https://jang.com.pk/news/1224322')]

In [ ]:
for column in events_all1.columns:
  print(column, type(events_all1[column][0]))

Event <class 'str'>
Sources <class 'list'>
Urls <class 'tuple'>
Articles <class 'int'>
Start Date <class 'pandas._libs.tslibs.timestamps.Timestamp'>
End Date <class 'pandas._libs.tslibs.timestamps.Timestamp'>
Description <class 'list'>
Categories <class 'list'>
Major_Cat <class 'str'>
Priority_cat <class 'str'>
Priority_Purity <class 'numpy.float64'>
Maj_Purity <class 'numpy.float64'>
Sentiment <class 'numpy.float64'>
Keywords <class 'list'>
Similarity_Scores <class 'list'>
NER_Persons <class 'list'>


In [ ]:
def fronzenset_to_tuple(s):
  s =s.replace("frozenset(","").replace(")", "").replace("'", "")
  return tuple(s.strip("{}").split(", "))

def convert_to_string(string):
  try:
    return ast.literal_eval(string)
  except (ValueError, SyntaxError):
    return string

## 5 Day Event Extraction

Take specific range data

In [19]:
date1 = '2023-03-01'
date2 = '2023-04-01'

# Filter the data for the year 2023
merged_Mar2023 = merged[(merged['DateTime'] >= date1) & (merged['DateTime'] <= date2)]

# Find the oldest and latest dates in 2023
oldest_date5 = merged_Mar2023['DateTime'].min()
latest_date5 = merged_Mar2023['DateTime'].max()

print("Oldest Date in 2023:", oldest_date5)
print("Latest Date in 2023:", latest_date5)

# Generate 5-day windows for the year 2023
dates5 = pd.date_range(start=oldest_date5, end=latest_date5, freq='D')
print(len(dates5), dates5)

Oldest Date in 2023: 2023-03-01 00:00:00
Latest Date in 2023: 2023-04-01 00:00:00
32 DatetimeIndex(['2023-03-01', '2023-03-02', '2023-03-03', '2023-03-04',
               '2023-03-05', '2023-03-06', '2023-03-07', '2023-03-08',
               '2023-03-09', '2023-03-10', '2023-03-11', '2023-03-12',
               '2023-03-13', '2023-03-14', '2023-03-15', '2023-03-16',
               '2023-03-17', '2023-03-18', '2023-03-19', '2023-03-20',
               '2023-03-21', '2023-03-22', '2023-03-23', '2023-03-24',
               '2023-03-25', '2023-03-26', '2023-03-27', '2023-03-28',
               '2023-03-29', '2023-03-30', '2023-03-31', '2023-04-01'],
              dtype='datetime64[ns]', freq='D')


Take just 2023 data

In [ ]:
import pandas as pd

# Filter the data for the year 2023
merged_2023 = merged[merged['DateTime'].dt.year == 2023]

# Find the oldest and latest dates in 2023
oldest_date5 = merged_2023['DateTime'].min()
latest_date5 = merged_2023['DateTime'].max()

print("Oldest Date in 2023:", oldest_date5)
print("Latest Date in 2023:", latest_date5)

# Generate 5-day windows for the year 2023
dates5 = pd.date_range(start=oldest_date5, end=latest_date5, freq='D')
print(len(dates5), dates5[0:5])


Oldest Date in 2023: 2023-01-01 00:00:00
Latest Date in 2023: 2023-08-18 00:00:00
230 DatetimeIndex(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04',
               '2023-01-05'],
              dtype='datetime64[ns]', freq='D')


In [20]:
%%time
import pandas as pd
import numpy as np

# # Find the oldest and latest dates
# oldest_date5 = merged['DateTime'].min()
# latest_date5 = merged['DateTime'].max()

# print("Oldest Date:", oldest_date5)
# print("Latest Date:", latest_date5)

# dates5 = list(merged['DateTime'].unique())
# # dates5 = np.sort(dates5)
# dates5 = pd.date_range(start=oldest_date5, end=latest_date5, freq='D')
# print(len(dates5), dates5[0:5])

# List of column names
cols = ['Event', 'Sources', 'Urls', 'Articles', 'Start Date', 'End Date']

# Create an empty dataframe to store all the events
global events
events = pd.DataFrame(columns=cols)

for i in range(4, len(dates5)):
    window_dates = dates5[i-4:i+1]  # Get the current 5-day window
    # Find the oldest and latest dates
    global start_date
    global end_date
    start_date = min(window_dates)
    end_date = max(window_dates)
    # print(f"Window: Dates: {window_dates}\n start: {start_date}\nend: {end_date}")
    # break
    df = merged[merged['DateTime'].isin(window_dates)]  # Filter dataframe within the window dates
    df.reset_index(drop=True, inplace=True)

    print(f"START_DATE: {start_date}, END_DATE: {end_date}")
    event_extraction(df)

events_all5 = events
events_all5.to_csv(f"/content/drive/MyDrive/Datasets/EventsExtracted/5_Day_Events_Mar2023.csv", index=False)

print(f"Columns: {events_all5.columns}\nShape: {events_all5.shape}")
events_all5.head(3)


START_DATE: 2023-03-01 00:00:00, END_DATE: 2023-03-05 00:00:00
START_DATE: 2023-03-02 00:00:00, END_DATE: 2023-03-06 00:00:00
START_DATE: 2023-03-03 00:00:00, END_DATE: 2023-03-07 00:00:00
START_DATE: 2023-03-04 00:00:00, END_DATE: 2023-03-08 00:00:00
START_DATE: 2023-03-05 00:00:00, END_DATE: 2023-03-09 00:00:00
START_DATE: 2023-03-06 00:00:00, END_DATE: 2023-03-10 00:00:00
START_DATE: 2023-03-07 00:00:00, END_DATE: 2023-03-11 00:00:00
START_DATE: 2023-03-08 00:00:00, END_DATE: 2023-03-12 00:00:00
START_DATE: 2023-03-09 00:00:00, END_DATE: 2023-03-13 00:00:00
START_DATE: 2023-03-10 00:00:00, END_DATE: 2023-03-14 00:00:00
START_DATE: 2023-03-11 00:00:00, END_DATE: 2023-03-15 00:00:00
START_DATE: 2023-03-12 00:00:00, END_DATE: 2023-03-16 00:00:00
START_DATE: 2023-03-13 00:00:00, END_DATE: 2023-03-17 00:00:00
START_DATE: 2023-03-14 00:00:00, END_DATE: 2023-03-18 00:00:00
START_DATE: 2023-03-15 00:00:00, END_DATE: 2023-03-19 00:00:00
START_DATE: 2023-03-16 00:00:00, END_DATE: 2023-03-20 0

,Event,Sources,Urls,Articles,Start Date,End Date,Description,Categories,Sub_Category,Sector Words,Major_Cat,Priority_cat,Priority_Purity,Maj_Purity,Sentiment,Keywords,Similarity_Scores,NER_Persons,NER_Orgs,NER_Loc
0,960 افراد کا بیک وقت کروشیا&nbsp; کرنے کا گین...,[Nawaiwaqt],[https://www.nawaiwaqt.com.pk/01-Mar-2023/1687...,1,2023-03-01,2023-03-05,[ڈربی(نیٹ نیوز) برطانیہ میں کروشیا کے شوقین 9...,[world],[NotAssigned],[],world,world,1.0,1.0,1.0,"[توڑا, افرد, افراد, 2017, گینیز, ریکارڈ, کروشی...",[-1],"[گینیز, کروشیا]",[],"[والا, برطانیہ]"
1,قابض فوج &nbsp;کی ریاستی &nbsp;دہشتگردی جاری م...,[Nawaiwaqt],[https://www.nawaiwaqt.com.pk/01-Mar-2023/1687...,1,2023-03-01,2023-03-05,[سری نگر(کے پی آئی) جنوبی کشمیر میں بھارتی فوج...,[world],[NotAssigned],[],world,world,1.0,1.0,0.0,"[سریتھ, فوجی, کشمیریوں, قتل, اونتی, مفتی, شہید...",[-1],"[مشتاق, شرما, سنجے, مودی, ؟محبوبہ, المجاہدین, ...",[پیپلز],"[پورہ, کشمیر, والا, ادھرضلع, نوشہرہ]"
2,امریکہ یمن کو انسانی ہمدردی کی بنیاد پر 40 &nb...,[Nawaiwaqt],[https://www.nawaiwaqt.com.pk/01-Mar-2023/1687...,1,2023-03-01,2023-03-05,[واشنگٹن (آئی این پی) امریکا نے یمن کو انسانی...,[world],[Economic],"[ارب, ڈالر]",world,world,1.0,1.0,1.0,"[ارب, بنیاد, جنیوا, انسانی, ہمدردی, ڈونرزکانفر...",[-1],"[امداد, بلنکن]",[متحدہ],"[واشنگٹن, امریکا, امریکہ]"


#### Duplicate Removal

In [21]:
events_all5['Urls'] = events_all5['Urls'].apply(tuple)

print(f"Shape Before Duplicate Removal: {events_all5.shape}")
# events_all1.drop_duplicates(inplace=True)
events_all5.drop_duplicates(subset='Urls' ,inplace=True)
events_all5.reset_index(drop=True)
print(".......................................")
print(f"Shape After Duplicate Removal: {events_all5.shape}")


indsub1, indsub2 = [], []
articles = np.sort(events_all5['Articles'].unique())
for art in articles:
  sub1 = events_all5[events_all5['Articles']==art]
  sub2 = events_all5[events_all5['Articles']>art]
  for index, row in sub1.iterrows():
    url = row['Urls']
    # print(type(url), url)
    for ind, row2 in sub2.iterrows():
      urls = row2['Urls']
      if set(url).issubset(set(urls)):
        indsub1.append(index)
        indsub2.append(ind)
  # print(art, sub1.shape, sub2.shape)

print(f"indeces dropped: {len(set(indsub1))}")

# Drop rows by indices
events_all5.drop(indsub1, inplace=True)

# Reset index after dropping rows
events_all5.reset_index(drop=True, inplace=True)

events_all5.to_csv("/content/drive/MyDrive/Datasets/EventsExtracted/5_Day_Events_Mar2023.csv", index=False)

# Print the resulting DataFrame
print(events_all5.shape)

Shape Before Duplicate Removal: (14685, 20)
.......................................
Shape After Duplicate Removal: (4224, 20)
indeces dropped: 1008
(3216, 20)


###### Extra

In [ ]:
articles = np.sort(events_all5['Articles'].unique())
articles

array([1, 2, 3, 4, 5, 6, 7, 8, 10, 12, 17], dtype=object)

In [ ]:
c = events_all5[events_all5['Articles']==8]
c.shape

(1, 16)

In [ ]:
d = list(c['Urls'])
d

[('https://jang.com.pk/news/1223866',
  'https://jang.com.pk/news/1223868',
  'https://www.nawaiwaqt.com.pk/09-May-2023/1704855',
  'https://jang.com.pk/news/1223846',
  'https://jang.com.pk/news/1223863',
  'https://jang.com.pk/news/1224334',
  'https://jang.com.pk/news/1225072',
  'https://www.nawaiwaqt.com.pk/13-May-2023/1705450')]